# [Advent of Code 2022 Day 14](https://adventofcode.com/2022/day/14/answer)
bested by sorites paradox...

In [ ]:
from __future__ import annotations
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2
run_doctest_for = lambda func: doctest.run_docstring_examples(func, globals())

## Test Cases

In [ ]:
PART_ONE_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 24,
    },
    "input": {
        "input1": 655,
    },
}
PART_ONE_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}
PART_ONE_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_ONE_CASES.keys()
}

In [ ]:
PART_TWO_CASES: dict[str, dict[str, str | int]] = {
    "example": {
        "example1": 93,
    },
    "input": {
        "input1": 26484,
    },
}
PART_TWO_INPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}
PART_TWO_OUTPUTS: dict[str, dict[str, str | int]] = {
    key: {} for key in PART_TWO_CASES.keys()
}

## Input Parsing

In [ ]:
class Model(BaseModel):
    data: Any

def line_to_points(line: str, ctx: Context | None = None) -> list[Point]:
    rock_coords = line.split(" -> ")
    if ctx is None:
        ctx = Context()
        ctx.x_min = INF
        ctx.x_max = -INF
        ctx.y_min = INF
        ctx.y_max = -INF
    parsed_coords = []
    for lmao in rock_coords:
        x, y = list(map(int, parse(r"(-?\d+),(-?\d+)", lmao)))
        ctx.x_min = min(ctx.x_min, x)
        ctx.x_max = max(ctx.x_max, x)
        ctx.y_min = min(ctx.y_min, y)
        ctx.y_max = max(ctx.y_max, y)
        parsed_coords.append(Point(x, y))
    return parsed_coords

def parse_input_from_filename(filename: str) -> Context:
    lines = list(yield_line(filename))

    ctx = Context()
    ctx.input = []

    input_lines = ctx.input

    ctx.x_min = INF
    ctx.x_max = -INF
    ctx.y_min = INF
    ctx.y_max = -INF

    for idx, line in enumerate(lines):
        input_lines.append(line_to_points(line, ctx))

    print(f"{ctx.x_min=} {ctx.x_max=} {ctx.y_min=} {ctx.y_max=}")

    return ctx

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["example"].keys() | PART_TWO_CASES["example"].keys())
def test_parsing_examples(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name", PART_ONE_CASES["input"].keys() | PART_TWO_CASES["input"].keys())
def test_parsing_inputs(test_file_name):
    for entity in parse_input_from_filename(test_file_name).input:
        enable_logging()
        log(f"{entity}")

## Helper Functions

### `get_intervals()`

In [ ]:
%%ipytest -xrP

def get_intervals(points: list[Point]) -> set[tuple[Point, Point]]:
    intervals = set()
    for i in range(len(points) - 1):
        point1 = points[i]
        point2 = points[i + 1]
        assert point1.adja_infinite(point2)
        intervals.add((point1, point2))
    return intervals

def test_helper_1() -> None:
    assert (points := get_intervals(line_to_points("498,4 -> 498,6 -> 496,6"))) == {
        (Point(498, 4), Point(498, 6)),
        (Point(498, 6), Point(496, 6)),
    }
    assert (points := get_intervals(line_to_points("503,4 -> 502,4 -> 502,9 -> 494,9"))) == {
        (Point(503, 4), Point(502, 4)),
        (Point(502, 4), Point(502, 9)),
        (Point(502, 9), Point(494, 9)),
    }

### `point_intersects_interval()`

In [ ]:
%%ipytest -xrPvvvvv

def point_intersects_interval(point: Point, interval: tuple[Point, Point]) -> bool:
    x, y = point
    point1, point2 = sorted(interval)
    if point1.horz_infinite(point2):
        return x == point1.x and point1.y <= y <= point2.y
    elif point1.vert_infinite(point2):
        return y == point1.y and point1.x <= x <= point2.x
    raise ValueError(f"Neither {point1=} nor {point2=} are parallel to each other")

def test_helper_2() -> None:
    assert point_intersects_interval(Point(498, 5), (Point(498, 4), Point(498, 6))) is True
    assert point_intersects_interval(Point(498, 4), (Point(498, 4), Point(498, 6))) is True
    assert point_intersects_interval(Point(498, 6), (Point(498, 4), Point(498, 6))) is True
    assert point_intersects_interval(Point(498, 7), (Point(498, 4), Point(498, 6))) is False
    assert point_intersects_interval(Point(498, 7), (Point(496, 7), Point(499, 7))) is True
    assert point_intersects_interval(Point(496, 7), (Point(496, 7), Point(499, 7))) is True
    assert point_intersects_interval(Point(499, 7), (Point(496, 7), Point(499, 7))) is True
    assert point_intersects_interval(Point(500, 7), (Point(496, 7), Point(499, 7))) is False

### `draw_board()`

In [ ]:
%%ipytest -xrPvvvvv

def draw_board(board: dict[Any, Any], x_min: int, x_max: int, y_min: int, y_max: int) -> None:
    for y in range(y_min, y_max + 1):
        for x in range(x_min, x_max + 1):
            log(f"{board[x][y]}", newline=False)
        log(f"")

def test_draw_board() -> None:
    draw_board(defaultdict(lambda: defaultdict(lambda: ".")), 0, 10, 0, 10)

### `initialize_board()`

In [ ]:
%%ipytest -xrPvvvvv

def initialize_board(board: dict[Any, Any], intervals: list[tuple[Point, Point]]) -> dict[Any, Any]:
    for pair in intervals:
        point1, point2 = sorted(pair)
        if point1.vert_infinite(point2):
            assert point1.y == point2.y
            for x in range(point1.x, point2.x + 1):
                board[x][point2.y] = "#"
        elif point1.horz_infinite(point2):
            assert point1.x == point2.x
            for y in range(point1.y, point2.y + 1):
                board[point1.x][y] = "#"
    return board

def test_initialize_board() -> None:
    board = defaultdict(lambda: defaultdict(lambda: "."))
    initialize_board(board, [(Point(0, 1), Point(0, 10))])
    draw_board(board, 0, 10, 0, 10)

### `drop_grain()`

In [ ]:
%%ipytest -xrPvvvvv

def drop_grain(ctx: Context, board: dict[Any, Any], drop: Point, max_timeout: int | None = None) -> bool:
    x, y = drop
    iterations = 0
    while True:
        iterations += 1
        if max_timeout is not None and iterations >= max_timeout:
            raise Exception("timed out")
        xx, yy = x, y + 1
        if y >= ctx.y_max + 2:
            return False
        loc = board[xx][yy]
        if loc == ".":
            log(f"{x, y} can drop to {xx, yy} because there is nothing below")
            x, y = xx, yy
        elif loc in "o#":
            log(f"{x, y} is blocked below, going to try and go left")
            # try to go left
            xx, yy = x - 1, y + 1
            loc = board[xx][yy]
            if loc in "o#":
                log(f"{x, y} is blocked to the left, going to try and go right")
                # try to go right
                xx, yy = x + 1, y + 1
                loc = board[xx][yy]
                if loc in "o#":
                    log(f"{x, y} is infinitely blocked. going to rest here forever.")
                    if board[x][y] == "o":
                        return False
                    board[x][y] = "o"
                    break
                else:
                    log(f"{x, y} can go right. doing that")
                    x, y = xx, yy
            else:
                log(f"{x, y} can go left. doing that")
                x, y = xx, yy
        else:
            raise NotImplementedError(f"For some reason... bad things happened...")
    return True

def test_helper_3() -> None:
    assert True

### `count_grains()`

In [ ]:
%%ipytest -xrPvvvvv

def count_grains(board: dict[Any, Any], x_min, x_max, y_min, y_max) -> int:
    cnt = 0
    for x in range(x_min, x_max + 1):
        for y in range(y_min, y_max + 1):
            if board[x][y] == "o":
                cnt += 1
    return cnt

## Main Function

In [ ]:
def solve(part: int, filename: str) -> int:
    ctx = parse_input_from_filename(filename)
    x_min, x_max, y_min, y_max = ctx.x_min, ctx.x_max, ctx.y_min, ctx.y_max
    input = ctx.input

    if part == 1:

        all_intervals = set()
        for points in input:
            all_intervals |= get_intervals(points)

        board = defaultdict(lambda: defaultdict(lambda: "."))
        board = initialize_board(board, all_intervals)
        disable_logging()
        while True:
            res = drop_grain(ctx, board, Point(500, 0))
            if not res:
                break

        return count_grains(board, x_min, x_max, y_min - 10, y_max)

    if part == 2:

        x_offset = x_min
        y_offset = 2

        all_intervals = set()
        for points in input:
            all_intervals |= get_intervals(points)
        all_intervals |= {(Point(x_min - x_offset, y_max + y_offset), Point(x_max + x_offset, y_max + y_offset))}
        x_min -= x_offset
        x_max += x_offset
        y_max += y_offset

        board = defaultdict(lambda: defaultdict(lambda: "."))
        board = initialize_board(board, all_intervals)
        disable_logging()
        while True:
            res = drop_grain(ctx, board, Point(500, 0))
            if not res:
                break

        return count_grains(board, x_min, x_max, y_min - 1000, y_max)
    else:
        raise Exception(f"Invalid part: {part}")

### Part 1

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["example"].items())
def test_part_one_examples(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES["input"].items())
def test_part_one_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(1, test_file_name)
    PART_ONE_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_ONE_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

### Part 2

In [ ]:
%%ipytest -xrPvvvvv
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["example"].items())
def test_part_two_examples(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["example"][test_file_name] = test_actual_output
    failure_message = "Did you forget to calibrate the example test case?" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed example test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES["input"].items())
def test_part_two_inputs(test_file_name, test_expected_output):
    test_actual_output = solve(2, test_file_name)
    PART_TWO_OUTPUTS["input"][test_file_name] = test_actual_output
    failure_message = f"Candidate answer {test_actual_output} found" if (
        test_expected_output == PART_TWO_SENTINEL
    ) else f"Failed input test case: expected {test_expected_output} but got {test_actual_output}"
    assert test_actual_output == test_expected_output, failure_message

dadada